In [ ]:
# import cell

import numpy as np

## einsum

- [numpy.sum](https://numpy.org/doc/stable/reference/generated/numpy.sum.html)
- [numpy.einsum](https://numpy.org/doc/stable/reference/generated/numpy.einsum.html)

`einsum`: Evaluates the Einstein summation convention on the operands.

Using the Einstein summation convention, many common multi-dimensional, linear algebraic array operations can be represented in a simple fashion. In implicit mode *einsum* computes these values.

The subscripts string is a <u>comma-separated</u> list of subscript labels, where each label refers to a dimension of the corresponding operand. Whenever a label is repeated it is summed.

---

[Understanding NumPy's einsum](https://stackoverflow.com/questions/26089893/understanding-numpys-einsum)

[A basic introduction to NumPy's einsum](https://ajcr.net/Basic-guide-to-einsum/) @ [1](http://www.atyun.com/32288.html)、[2](https://blog.popkx.com/A-basic-introduction-to-NumPy-s-einsum/)  
[Einsum Is All You Need: NumPy, PyTorch and TensorFlow](https://www.youtube.com/watch?v=pkVwUVEHmfI)  
[EINSUM IS ALL YOU NEED - EINSTEIN SUMMATION IN DEEP LEARNING](https://rockt.github.io/2018/04/30/einsum)  

[一个函数打天下，einsum](https://zhuanlan.zhihu.com/p/71639781)  
[如何理解和使用NumPy.einsum？](https://zhuanlan.zhihu.com/p/27739282)  
[Python 學習筆記｜numpy.einsum用法](https://medium.com/programming-with-data/python-%E5%AD%B8%E7%BF%92%E7%AD%86%E8%A8%98-numpy-einsum%E7%94%A8%E6%B3%95-2169af18c475)  


### implicit mode

`np.einsum('i,i', a, b)` is equivalent to `np.inner(a,b)`.

- 单字母下标只能索引1D vector，将a和b对应位置元素相乘再求和，即求点/内积。

`np.einsum('ij,ij', a, b)` is equivalent to `tensordot(a,b,axes=([0,1],[0,1]))`。

- 两个二维索引名称和顺序完全一致，表示相同维度相同位置相乘再求和（shape overlap multiply sum）。

`np.einsum('ij,jk', a, b)` describes traditional matrix multiplication and is equivalent to `np.matmul(a,b)`.

- 交汇索引`j`为a的第二维、b的第一维，相当于 `tensordot(axes=(1,0))`，等价于普通的矩阵乘法。

![matmul_reduce](https://ajcr.net/images/matrix_mul_reduce.png)

`np.einsum('ij,jh', a, b)` returns the **transpose** of the multiplication since subscript ‘h’ precedes subscript ‘i’.

- 正常脚标顺序的表达式 np.einsum('ij,jk', a, b) 计算结果为 `ik`，此处结果为 `ih`，相当于对结果进行转置。

`np.einsum('ii', a)` is equivalent to `np.trace(a)`.

- `ii` 表示横纵索引一样，相当于对角线元素相加，整体等价于 trace。

`np.einsum('ij', a)` doesn’t affect a 2D array, while `np.einsum('ji', a)` takes its **transpose**.

- `ji` 将横纵坐标调换顺序，相当于求其转置。


In [ ]:
# vector einsum

a=np.arange(1,5)
b=np.arange(5,9)
es_dot = np.einsum('i,i', a, b)
print('es_dot:\n', es_dot)

# 对角线相加(trace)
c = np.arange(9).reshape(3,3)
print('c.diagnoal():\n', c.diagonal())
es_trace = np.einsum('ii', c)
print('es_trace:\n', es_trace)

# 求转置
es_T = np.einsum('ji', c)
print('es_T:\n', es_T)


In [ ]:
# 2D einsum

# sum all elements
a = np.arange(6).reshape(2, 3)
es_sum = np.einsum('ij->', a)
print('es_sum:\n', es_sum)

# sum columns
es_sum_col = np.einsum('ij->j', a)
print('es_sum_col:\n', es_sum_col)

# sum rows
es_sum_row = np.einsum('ij->i', a)
print('es_sum_row:\n', es_sum_row)


In [ ]:
# 2D einsum

a = np.array([1,2,3,4]).reshape(2,2)
b = np.array([5,6,7,8]).reshape(2,2)
es_vdot = np.einsum('ij,ij', a, b)
print('es_vdot:\n', es_vdot)
es_inner = np.einsum('ij,jk', a, b)
print('es_inner:\n', es_inner)


### explicit mode

In explicit mode the output can be directly controlled by specifying output subscript labels. This requires the identifier ‘`->`’ as well as the list of output subscript labels. This feature increases the flexibility of the function since summing can be disabled or forced when required.

The call `np.einsum('i->', a)` is like `np.sum(a, axis=-1)`, and `np.einsum('ii->i', a)` is like `np.diag(a)`. The difference is that einsum **does not** allow broadcasting by default.

Additionally `np.einsum('ij,jh->ih', a, b)` directly specifies the order of the output subscript labels and therefore returns matrix multiplication, unlike the example above in implicit mode.

In [ ]:
# explicit

# 单字母小标对应1D vector
a = np.array([1,2,3,4])
es1 = np.einsum('i->', a)
print('es1:\n', es1)

b = np.array([5,6,7,8]).reshape(2,2)
# ValueError: operand has more dimensions than subscripts given
# es2 = np.einsum('i->', b)
# ellipsis provided to broadcast the extra dimensions.
es2 = np.einsum('...i->...', b)
print('es2:\n', es2)

# 对角线向量，二维索引变一维索引
es3 = np.einsum('ii->i', b)
print('es3:\n', es3)

# 将a变成2D
a = np.array([1,2,3,4]).reshape(2,2)
es4 = np.einsum('ij,jh->ih', a, b)
print('es4:\n', es4)

In [ ]:
# examples

# multiply element-wise and then sum along axis 1 (the rows of the array)
a = np.array([0, 1, 2])
b = np.array([[ 0,  1,  2,  3],
              [ 4,  5,  6,  7],
              [ 8,  9, 10, 11]])
c = (a[:, np.newaxis] * b).sum(axis=1)
c = (np.expand_dims(a, axis=1) * b).sum(axis=1)
c = np.einsum('i,ij->i', a, b)


### tensor contraction

a = $ \begin{bmatrix} 0 & 1 \\ 2 & 3 \\ 4 & 5 \\ \end{bmatrix} $，
b = $ \begin{bmatrix} 0 & 1 & 2 \\ 3 & 4 & 5 \\ 6 & 7 & 8 \\ 9 & 10 & 11 \\ \end{bmatrix} $

`np.einsum('ki,jk->ij', a, b)`：相同脚标`k`的位置为a的第一维（列方向）、b的第二维（行方向），  
相当于 np.tensordot(a,b,axes=(0,1)) 或 a.T @ b.T （np.matmul(a.T, b.T）。

参考 [numpy中的tensordot](https://www.cnblogs.com/traditional/p/12639487.html)，其中提供了一些 tensordot 等效的 einsum 表达式写法。

In [ ]:
# tensor contraction

# 2D · 2D
a = np.arange(6).reshape((3,2))
b = np.arange(12).reshape((4,3))
es1 = np.einsum('ki,jk->ij', a, b)
print('es1:\n', es1)
# 等效于指定a第一维和b第二维进行tensordot，结果shape=(2,4)
tdot1 = np.tensordot(a,b,axes=(0,1))
print('tdot1:\n', tdot1)

# 3D · 3D
c = np.arange(60.).reshape(3,4,5)
d = np.arange(24.).reshape(4,3,2)
es2 = np.einsum('ijk,jil->kl', c, d)
print('es2:\n', es2)
# 等效于指定c第一、二维和d第二、一维进行tensordot，结果shape=(5,2)
tdot2 = np.tensordot(c,d,axes=([0,1],[1,0]))
print('tdot2:\n', tdot2)
